In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import gmaps
import os
import requests
import shutil

In [13]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [10]:
input_folder = '../data/railways/global/'
input_file = input_folder+'australia-oceania_main.csv'

output_folder = '../data/output_images/Australia/'
output_csv = output_folder+'AUS.csv'

### Import Routes

In [3]:
routes = pd.read_csv(input_folder)


routes.head()

,Way ID (track segment),Node ID,Latitude,Longitude,Elevation(feet)
0,2654260,11978242,-33.868837,151.196420,36.1
1,2654260,11978239,-33.868756,151.196246,36.1
2,2761538,1570660892,-33.879133,151.201152,32.8
3,2761538,1855173811,-33.879122,151.201066,32.8
4,2761538,5016259402,-33.879112,151.201002,32.8


In [4]:
# Number of routes
routes.shape

(557284, 5)

In [5]:
'''
Get all points
'''

# len(routes['Node ID'].tolist())

points = []
for i in range(0,1000):
    try:
        route = routes.iloc[i]
        sec_points = (route['Longitude'],route['Latitude'])
        points.append(sec_points)
    except Exception as e:
        print("Skipped route number",i,'because',e)

print(len(points),'points')

1000 points


In [ ]:
'''
Get points for 1 or more routes
'''

rows = [0]

points = []
for i in range(0,len(routes.OBJECTID.tolist())):
    if i in rows:
    #     print('Route',i)
        try:
            route = routes.iloc[i]
            g = [m for m in route.geometry]
    #         print('\t',len(g),'sections')
            for j in range(0,len(g)):
    #             print('   section',j)
                sec_points = mapping(g[j])["coordinates"]
                for k in range(0,len(sec_points)):
    #                 print('      point',k)
                    points.append(sec_points[k])
        except Exception as e:
            print("Skipped route number",i,'because',e)

print(len(points),'points')

In [ ]:
'''
Get points for single route/section
'''

row = 0
section = 0

points = []
try:
    route = routes.iloc[row]
    g = [i for i in route.geometry]
    len(g)

    points = mapping(g[section])["coordinates"]
except Exception as e:
    print("Skipped route number",row,'because',e)
    
print(len(points),'points')

In [6]:
'''
Get subselection of points
'''

route_points = []
for point in range(0,len(points),20):
    tmp = points[point]
    route_points.append(tmp)

print(len(route_points))

50


In [7]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]


df.head()

,Name,Longitude,Latitude
0,151.1964198_-33.868837,151.196420,-33.868837
1,151.2070754_-33.8801401,151.207075,-33.880140
2,151.9480466_-27.573679499999997,151.948047,-27.573679
3,153.0118785_-27.5267477,153.011878,-27.526748
4,153.00315669999998_-27.5259447,153.003157,-27.525945


In [9]:
'''
Export points
'''

df.to_csv(output_csv,header=True,index=None)

In [10]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))
    
# Set map centerpoint
coords = marker_points[0]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 13
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [12]:
'''
Get satellite preview for image
'''

row = 0

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

Figure(layout=FigureLayout(height='800px', width='100%'))

In [15]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Africa.ipynb'))
image_folder = root[:-9]+output_folder[3:]
image_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
# url = 'https://www.google.com/maps/@?api=1&map_action=map'
rows = df.index.tolist()
for i in range(0,len(rows)):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 19,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = image_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 